In [42]:
import os
import glob
import pandas as pd
import logging
import numpy as np
import re
from tqdm import tqdm

In [40]:
os.chdir("/data/localhost/taufiq/pulse-data")
extension = 'final_.csv'
all_filenames = [i for i in glob.glob('*{}'.format(extension))]
pulse_data = pd.concat([pd.read_csv(f) for f in all_filenames ])

In [75]:
pulse_data['icustay_id'] = pulse_data['id'].astype(int)
pulse_data.drop(columns='id', inplace=True)
pulse_data = pulse_data.reset_index(drop=False)
pulse_data.head()

,index,Albumin,paCO2,paO2,ArterialPressure (mmHg),HCO3,Arterial_pH,BloodUreaNitrogenConcentration (mg/dL),BloodVolume (mL),Calcium,CarbonDioxideSaturation (None),CardiacOutput (L/min),Chloride,Creatinine,DiaBP,EndTidalCarbonDioxidePressure (mmHg),Glucose,HR,HemoglobinContent (g),Arterial_lactate,MeanBP,OxygenSaturation (None),Potassium,RR,PaO2_FiO2,SimulationTime(s),Temp_C,Sodium,SysBP,WBC_count,time,icustay_id
0,0,36949.438638,40.082410,91.855017,98.292663,1587.765562,7.412865,13.059566,5601.571764,48.187032,0.028649,9.332884,3627.397096,15.017193,61.581876,33.516288,1007.745301,115.384615,760.838266,145.623973,91.201544,0.975979,156.391530,20.000000,4.647789,0.0,33.279228,3330.266997,118.768598,7000.0,7.103817e+09,75585
1,1,35975.628338,40.314134,92.062561,60.025035,1587.686746,7.410586,15.328943,5741.443742,53.133540,0.028530,9.506198,3492.500049,13.609476,60.025077,33.715665,1871.240876,111.111111,760.858331,130.026189,91.397051,0.975675,152.325811,20.000000,4.646387,3600.0,35.056723,3318.178469,121.036521,7000.0,7.103820e+09,75585
2,0,36941.846898,39.893587,88.507821,87.414676,1590.467389,7.414731,13.081810,5911.672498,48.154955,0.028717,5.346963,3625.571755,14.974856,60.469035,34.515789,953.853188,65.217391,903.193845,145.773336,91.925755,0.973469,156.361298,15.544041,4.635947,0.0,33.079131,3322.906770,121.156909,7000.0,6.412648e+09,1528
3,1,35936.144876,40.085716,90.793879,123.365319,1590.964542,7.412832,15.169833,6064.724204,52.697856,0.028483,5.525505,3488.744953,13.552795,60.080616,34.836578,1524.323126,63.829787,903.186663,137.523405,93.070532,0.973862,152.167101,16.216216,4.637655,3600.0,31.448060,3312.145317,123.632804,7000.0,6.412651e+09,1528
4,0,36968.735379,39.894300,91.729049,120.522155,1589.726134,7.414724,13.066101,4691.617731,48.192895,0.028700,6.386956,3627.891455,14.946875,63.658505,34.089725,995.029514,81.081081,717.293539,145.503546,93.621503,0.975919,156.469415,19.867550,4.649578,0.0,33.144292,3327.133490,120.941327,7000.0,6.551858e+09,3198


In [56]:
column_mappings = {
    'Albumin - BloodConcentration (mg/L)': 'Albumin',
    'ArterialCarbonDioxidePressure (mmHg)': 'paCO2',
    'ArterialOxygenPressure (mmHg)': 'paO2',
    'Bicarbonate - BloodConcentration (mg/L)': 'HCO3',
    'BloodPH (None)': 'Arterial_pH',
    'Calcium - BloodConcentration (mg/L)': 'Calcium',
    'Chloride - BloodConcentration (mg/L)': 'Chloride',
    'Creatinine - BloodConcentration (mg/L)': 'Creatinine',
    'DiastolicArterialPressure (mmHg)': 'DiaBP',
    'Glucose - BloodConcentration (mg/L)': 'Glucose',
    'Lactate - BloodConcentration (mg/L)': 'Arterial_lactate',
    'MeanArterialPressure (mmHg)': 'MeanBP',
    'Potassium - BloodConcentration (mg/L)': 'Potassium',
    'RespirationRate (1/min)': 'RR',
    'SaturationAndFractionOfInspiredOxygenRatio (None)': 'PaO2_FiO2',
    'SkinTemperature (degC)': 'Temp_C',
    'Sodium - BloodConcentration (mg/L)': 'Sodium',
    'SystolicArterialPressure (mmHg)': 'SysBP',
    'WhiteBloodCellCount (ct/uL)': 'WBC_count',
    'HeartRate (1/min)': 'HR'
}

pulse_rename = {}

for k, v in column_mappings.items():
    pulse_rename.update({k: f"{v}"})

pulse_data = pulse_data.rename(columns=pulse_rename)

In [22]:
len(pulse_data)

9894

In [59]:
times = [10.02, 3600.02]
biogears_data = biogears_data[biogears_data['Time(s)'].isin(times)]
biogears_data['icustay_id'] = biogears_data['icustay_id'].astype(int)
icustayids = []
for icustay_id in biogears_data['icustay_id'].unique():
    if (biogears_data['icustay_id'] == icustay_id).sum() == 2:
        icustayids.append(icustay_id)
        
biogears_data = biogears_data[biogears_data['icustay_id'].isin(icustayids)].reset_index(drop=True)

column_mappings = {
    'Albumin-BloodConcentration(ug/mL)': 'Albumin',
    'ArterialCarbonDioxidePressure(mmHg)': 'paCO2',
    'ArterialOxygenPressure(mmHg)': 'paO2',
    'Bicarbonate-BloodConcentration(ug/mL)': 'HCO3',
    'ArterialBloodPH': 'Arterial_pH',
    'Calcium-BloodConcentration(ug/mL)': 'Calcium',
    'Chloride-BloodConcentration(ug/mL)': 'Chloride',
    'Creatinine-BloodConcentration(ug/mL)': 'Creatinine',
    'DiastolicArterialPressure(mmHg)': 'DiaBP',
    'Glucose-BloodConcentration(ug/mL)': 'Glucose',
    'Lactate-BloodConcentration(ug/mL)': 'Arterial_lactate',
    'MeanArterialPressure(mmHg)': 'MeanBP',
    'Potassium-BloodConcentration(ug/mL)': 'Potassium',
    'RespirationRate(1/min)': 'RR',
    'SkinTemperature(degC)': 'Temp_C',
    'Sodium-BloodConcentration(ug/mL)': 'Sodium',
    'SystolicArterialPressure(mmHg)': 'SysBP',
    'WhiteBloodCellCount(ct/uL)': 'WBC_count',
    'HeartRate(1/min)': 'HR'
}

biogears_rename = {}

for k, v in column_mappings.items():
    biogears_rename.update({k: f"{v}"})

biogears_data = biogears_data.rename(columns=biogears_rename)

In [43]:
os.chdir("/data/localhost/taufiq/bigears-data")
extension = '.csv'
all_filenames = [i for i in glob.glob('*{}'.format(extension))]
biogears_data = pd.DataFrame()
for f in tqdm(all_filenames):
    if os.path.getsize(f) > 0:
        df = pd.read_csv(f)
        m = re.search('SimulateMIMIC_(.+?)_.csv', f)
        if m:
            icustay_id = m.group(1)
            df['icustay_id'] = icustay_id
        biogears_data = biogears_data.append(df, ignore_index=True)

100%|██████████| 6476/6476 [21:26<00:00,  5.03it/s]


In [51]:
biogears_data.drop(columns=['Piperacillin-BloodConcentration', 'UrinationRate(mL/min)', 'Tazobactam-BloodConcentration', 'Magnesium-BloodConcentration'], inplace=True)
pd.set_option('display.max_columns', None)
biogears_data.head()

,Time(s),HR,MeanBP,SysBP,DiaBP,CardiacOutput(mL/min),HemoglobinContent(g),CentralVenousPressure(mmHg),Hematocrit,Arterial_pH,WBC_count,UrineProductionRate(mL/s),RR,OxygenSaturation,CarbonDioxideSaturation,CoreTemperature(degC),Temp_C,HCO3,Creatinine,Arterial_lactate,Glucose,Sodium,Potassium,Chloride,Calcium,Albumin,icustay_id
0,10.02,62.2313,90.1170,117.505,61.2284,5247.67,469.204,4.29491,0.459228,7.40294,6989.47,0.021750,19.2308,0.982376,0.028252,36.9894,32.7513,1592.86,10.54140,77.7979,819.333,3243.29,160.821,3777.32,94.5324,44957.8,69184
1,3600.02,58.5146,91.3791,122.405,58.9555,5685.91,469.211,4.69330,0.424163,7.44205,6455.78,0.038167,19.2308,0.981464,0.030803,36.7697,32.4991,1643.62,8.13846,85.0513,738.848,3207.18,157.634,3662.20,91.3362,41539.1,69184
2,10.02,86.5803,101.8150,120.811,80.8178,7196.23,1100.620,4.21137,0.460022,7.46220,7001.57,0.032391,18.5185,0.984664,0.030225,37.0185,33.2255,1589.81,10.92650,79.2878,842.446,3222.66,159.813,3767.98,93.9551,44989.0,38346
3,3600.02,84.5227,102.3460,122.684,79.7390,7595.43,1100.630,4.41927,0.442152,7.49379,6729.58,0.039467,18.1818,0.984477,0.031874,37.2043,33.8567,1617.82,9.55419,88.4465,757.909,3194.33,157.315,3689.29,91.8278,43073.5,38346
4,10.02,83.2297,98.1942,121.029,73.2544,7047.01,765.027,4.15863,0.459815,7.40575,6998.41,0.005003,16.7598,0.980015,0.028358,37.0014,33.1367,1591.22,10.88400,79.6498,854.989,3239.68,160.122,3784.06,94.5441,44990.7,16224


In [60]:
biogears_data.to_csv("/data/localhost/taufiq/export-dir/BiogearsMIMIC.csv")

In [112]:
MIMICtable = pd.read_csv("/data/localhost/taufiq/export-dir/MIMIC-1hourly-length-2.csv")
MIMICtable['icustay_id'] = MIMICtable['icustay_id'].astype(int)
MIMICtable.head()

,bloc,icustay_id,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,delay_end_of_record_and_discharge_or_death,SOFA,SIRS,Weight_kg,GCS,HR,SysBP,MeanBP,DiaBP,RR,SpO2,Temp_C,FiO2_1,Potassium,Sodium,Chloride,Glucose,BUN,Creatinine,Magnesium,Calcium,Ionised_Ca,CO2_mEqL,SGOT,SGPT,Total_bili,Albumin,Hb,WBC_count,Platelets_count,PTT,PT,INR,Arterial_pH,paO2,paCO2,Arterial_BE,HCO3,Arterial_lactate,mechvent,Shock_Index,PaO2_FiO2,median_dose_vaso,max_dose_vaso,input_total,input_1hourly,output_total,output_1hourly,cumulated_balance
0,1,65537,7.139723e+09,0.0,22799.404780,4.0,1.0,0.0,0.0,0.0,61.083333,10.0,1,107.000000,14.0,77.0,112.0,81.0,58.0,14.000000,99.600000,35.833333,0.36,187.671840,3126.608584,3155.05,1190.0,25.0,10.0,2.2,92.0,1.25,44.0,7027.0,9830.0,14.0,34000.0,11.769392,11500.0,269.0,73.30,13.4,1.10,7.35,131.0,40.0,-3.000000,1098.3024,243.216,0.0,0.687500,363.888889,0.18,0.18,4020.493550,29.994183,7235.0,3605.0,-3184.512267
1,2,65537,7.139727e+09,0.0,22799.404780,4.0,1.0,0.0,0.0,0.0,61.083333,0.0,1,98.600000,15.0,70.5,117.0,79.0,63.0,18.833333,99.666667,35.833333,0.36,173.335797,3126.608584,3651.35,1560.0,11.0,5.0,1.9,84.0,1.19,17.0,14.0,15.0,0.4,27000.0,9.917540,9000.0,162.0,26.70,19.0,1.80,7.31,155.0,33.0,-4.666667,1769.4872,243.216,0.0,0.602564,430.555556,0.00,0.00,4050.487733,515.407667,10760.0,3525.0,-6194.104600
2,1,3,7.245400e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,57.333333,5.0,2,66.300000,10.0,80.0,120.5,83.5,65.0,24.000000,98.500000,37.444445,0.40,0.000000,2942.690432,3509.55,1040.0,50.0,18.0,1.9,80.0,1.08,24.0,92.0,82.0,3.4,21000.0,10.400000,19200.0,150.0,36.25,17.0,1.55,7.30,240.0,46.0,-3.000000,1891.5208,324.288,0.0,0.663900,599.999991,0.00,0.00,6267.000000,0.000000,9490.0,4705.0,-3223.000000
3,2,3,7.245403e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,57.333333,3.0,1,78.699997,10.0,79.0,117.5,82.5,64.0,21.500000,98.500000,37.444445,0.40,0.000000,3241.557429,3828.60,1040.0,8.0,6.0,2.1,75.0,1.01,30.0,26.0,32.0,0.6,33000.0,9.900000,6500.0,211.0,26.50,12.0,1.00,7.41,136.0,35.0,1.000000,1647.4536,90.080,0.0,0.672340,339.999995,0.00,0.00,6267.000000,10.000000,14095.0,4605.0,-7818.000000
4,1,65544,7.056917e+09,0.0,24000.436400,3.0,0.0,0.0,0.0,0.0,22.333333,0.0,1,73.900002,15.0,87.5,113.5,70.5,48.5,25.500000,95.000000,37.111113,0.44,164.212860,3172.588122,3686.80,1270.0,7.0,6.0,1.9,65.0,1.16,28.0,40.0,26.0,0.2,51000.0,13.300000,9300.0,343.0,23.00,19.0,1.80,7.39,413.0,44.0,1.000000,1708.4704,72.064,0.0,0.770925,938.636364,0.00,0.00,11332.000000,0.000000,8210.0,4105.0,3122.000000


In [94]:
from sklearn.cluster import KMeans
from scipy.stats import rankdata

nra = 5

logging.info('Creating action bins')
nact = nra**2
input_1hourly_nonzero = MIMICtable.loc[MIMICtable['input_1hourly']>0, 'input_1hourly']
iol_ranked = rankdata(input_1hourly_nonzero)/len(input_1hourly_nonzero) # excludes zero fluid (will be action 1)
iof = np.floor((iol_ranked + 0.2499999999)*4) # converts iv volume in 4 actions
io = np.ones(len(MIMICtable)) # array of ones, by default
io[MIMICtable['input_1hourly']>0] = iof + 1 # where more than zero fluid given: save actual action
vc = MIMICtable['max_dose_vaso'].copy()
vc_nonzero = MIMICtable.loc[MIMICtable['max_dose_vaso']!=0, 'max_dose_vaso']
vc_ranked = rankdata(vc_nonzero)/len(vc_nonzero)
vcf = np.floor((vc_ranked + 0.2499999999)*4) # converts to 4 bins
vcf[vcf==0] = 1
vc[vc!=0] = vcf + 1
vc[vc==0] = 1
# median dose of drug in all bins
ma1 = [MIMICtable.loc[io==1, 'input_1hourly'].median(), MIMICtable.loc[io==2, 'input_1hourly'].median(), MIMICtable.loc[io==3, 'input_1hourly'].median(), MIMICtable.loc[io==4, 'input_1hourly'].median(), MIMICtable.loc[io==5, 'input_1hourly'].median()]
ma2 = [MIMICtable.loc[vc==1, 'max_dose_vaso'].median(), MIMICtable.loc[vc==2, 'max_dose_vaso'].median(), MIMICtable.loc[vc==3, 'max_dose_vaso'].median(), MIMICtable.loc[vc==4, 'max_dose_vaso'].median(), MIMICtable.loc[vc==5, 'max_dose_vaso'].median()]
med = pd.DataFrame(data={'IV':io, 'VC': vc})
med = med.astype({'IV': 'int32', 'VC': 'int32'})
uniqueValues = med.drop_duplicates().reset_index(drop=True)
uniqueValueDoses = pd.DataFrame()
for index, row in uniqueValues.iterrows():
    uniqueValueDoses.at[index, 'IV'], uniqueValueDoses.at[index, 'VC'] = ma1[row['IV']-1], ma2[row['VC']-1]

actionbloc = pd.DataFrame()
for index, row in med.iterrows():
    actionbloc.at[index, 'action_bloc'] = uniqueValues.loc[(uniqueValues['IV'] == row['IV']) & (uniqueValues['VC'] == row['VC'])].index.values[0]+1
actionbloc = actionbloc.astype({'action_bloc':'int32'})

logging.info('Action bins created')

In [124]:
MIMICtable['A_t'] = actionbloc['action_bloc']-1
MIMICtable_filtered = MIMICtable[MIMICtable['icustay_id'].isin(pulse_data['icustay_id'])]

In [125]:
cols = ['bloc', 'gender', "age", 'Weight_kg', 'icustay_id', 'Temp_C', 'HR', 'SysBP', 'MeanBP', 'DiaBP', 'RR',
       'Potassium', 'Sodium', 'Chloride', 'Glucose', 'Creatinine', 'Calcium', 
       'Albumin', 'WBC_count', 'Arterial_pH',
       'HCO3', 'Arterial_lactate', 'A_t']
MIMICtable_filtered = MIMICtable_filtered[cols]

In [126]:
#################   Convert training data and compute conversion factors    ######################
logging.info('Processing raw data')
# all 47 columns of interest

colbin = ['mechvent','re_admission']
colnorm=['GCS','HR','SysBP','MeanBP','DiaBP','RR','Temp_C','FiO2_1',\
    'Potassium','Sodium','Chloride','Glucose','Magnesium','Calcium',\
    'Hb','WBC_count','Platelets_count','PTT','PT','Arterial_pH','paO2','paCO2',\
    'Arterial_BE','HCO3','Arterial_lactate','SOFA','SIRS','Shock_Index','PaO2_FiO2','Albumin']
collog=['SpO2','BUN','Creatinine','SGOT','SGPT','Total_bili','INR','output_total','output_1hourly']

MIMICraw = MIMICtable_filtered.copy()
pulse_raw = pulse_data.copy()
biogears_raw = biogears_data.copy()

for col in MIMICraw:
    if col == 'gender':
        MIMICraw[col] = MIMICraw[col] - 0.5
    elif col in ['Weight_kg', 'age']:
        cmu = MIMICraw[col].mean()
        csigma = MIMICraw[col].std()
        MIMICraw[col] = (MIMICraw[col] - cmu)/csigma
    if col in colbin:
        MIMICraw[col] = MIMICraw[col] - 0.5
        pulse_raw[col] = pulse_data[col] - 0.5
        biogears_raw[col] = biogears_data[col] - 0.5
    elif col in colnorm:
        cmu = MIMICraw[col].mean()
        csigma = MIMICraw[col].std()
        MIMICraw[col] = (MIMICraw[col] - cmu)/csigma
        pulse_raw[col] = (pulse_data[col] - cmu)/csigma
        biogears_raw[col] = (biogears_data[col] - cmu)/csigma
    elif col in collog:
        log_values = np.log(0.1 + MIMICraw[col])
        dmu = log_values.mean()
        dsigma = log_values.std()
        MIMICraw[col] = (log_values - dmu)/dsigma
        pulse_raw[col] = (np.log(0.1 + pulse_data[col]) - dmu)/dsigma
        biogears_raw[col] = (np.log(0.1 + biogears_data[col]) - dmu)/dsigma
logging.info('Raw data processed')

for col in pulse_raw.columns:
    if col not in MIMICraw:
        cmu = pulse_raw[col].mean()
        csigma = pulse_raw[col].std()
        pulse_raw[col] = (pulse_raw[col] - cmu)/csigma

for col in biogears_raw.columns:
    if col not in MIMICraw:
        cmu = biogears_raw[col].mean()
        csigma = biogears_raw[col].std()
        biogears_raw[col] = (biogears_raw[col] - cmu)/csigma        

In [127]:
len(biogears_raw)

7400

In [128]:
icustayids = set(pulse_raw['icustay_id']).intersection(set(biogears_raw['icustay_id']))
biogears_raw = biogears_raw[biogears_raw['icustay_id'].isin(icustayids)]
biogears_data = biogears_data[biogears_data['icustay_id'].isin(icustayids)]
pulse_raw = pulse_raw[pulse_raw['icustay_id'].isin(icustayids)]
pulse_data = pulse_data[pulse_data['icustay_id'].isin(icustayids)]
MIMICraw = MIMICraw[MIMICraw['icustay_id'].isin(icustayids)]
MIMICtable_filtered = MIMICtable_filtered[MIMICtable_filtered['icustay_id'].isin(icustayids)]

In [129]:
biogears_raw.to_csv("/data/localhost/taufiq/mimic-pls-data/biogears_raw.csv")
pulse_raw.to_csv("/data/localhost/taufiq/mimic-pls-data/pulse_raw.csv")
biogears_data.to_csv("/data/localhost/taufiq/mimic-pls-data/biogears_data.csv")
pulse_data.to_csv("/data/localhost/taufiq/mimic-pls-data/pulse_data.csv")
MIMICraw.to_csv("/data/localhost/taufiq/mimic-pls-data/mimic_raw.csv")
MIMICtable_filtered.to_csv("/data/localhost/taufiq/mimic-pls-data/mimic_data.csv")

In [133]:
biogears_data.columns

Index(['Time(s)', 'HR', 'MeanBP', 'SysBP', 'DiaBP', 'CardiacOutput(mL/min)',
       'HemoglobinContent(g)', 'CentralVenousPressure(mmHg)', 'Hematocrit',
       'Arterial_pH', 'WBC_count', 'UrineProductionRate(mL/s)', 'RR',
       'OxygenSaturation', 'CarbonDioxideSaturation', 'CoreTemperature(degC)',
       'Temp_C', 'HCO3', 'Creatinine', 'Arterial_lactate', 'Glucose', 'Sodium',
       'Potassium', 'Chloride', 'Calcium', 'Albumin', 'icustay_id'],
      dtype='object')

In [130]:
actionbloc['action_bloc'].max()

25